In [189]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import yfinance as yf
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import pandas_ta as ta
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [190]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [191]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]

In [192]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,5,15)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# hour4 df
timeframe = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# daily df
timeframe = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
daily_df
hourly_df
ohlc_df

,time,open,high,low,close
0,2025-05-14 23:00:00,1.11674,1.11693,1.11672,1.11687
1,2025-05-14 23:01:00,1.11687,1.11687,1.11666,1.11670
2,2025-05-14 23:02:00,1.11670,1.11680,1.11670,1.11676
3,2025-05-14 23:03:00,1.11676,1.11681,1.11671,1.11673
4,2025-05-14 23:04:00,1.11673,1.11674,1.11666,1.11667
...,...,...,...,...,...
53281,2025-07-04 23:52:00,1.17762,1.17766,1.17718,1.17721
53282,2025-07-04 23:53:00,1.17721,1.17722,1.17719,1.17722
53283,2025-07-04 23:54:00,1.17722,1.17722,1.17697,1.17713
53284,2025-07-04 23:55:00,1.17713,1.17716,1.17711,1.17716


In [193]:
#ohlc4
daily_df['ohlc'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
hourly_df['ohlc'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4
#sma
daily_df['sma_1'] = ta.sma(close = daily_df['ohlc'], length= 1 )
daily_df['sma_14'] = ta.sma(close = daily_df['ohlc'], length= 14)
hourly_df['sma_11'] = ta.sma(close = hourly_df['ohlc'], length= 1)
hourly_df['sma_144'] = ta.sma(close = hourly_df['ohlc'], length= 14)
daily_df['high_20'] = ta.sma(close = daily_df['high'], length= 20 )
daily_df['low_20'] = ta.sma(close = daily_df['low'], length= 20)
daily_df

,time,open,high,low,close,ohlc,sma_1,sma_14,high_20,low_20
0,2025-05-15,1.11700,1.12280,1.11668,1.11835,1.118707,1.118707,NaN,NaN,NaN
1,2025-05-16,1.11866,1.12197,1.11312,1.11649,1.117560,1.117560,NaN,NaN,NaN
2,2025-05-19,1.11755,1.12884,1.11694,1.12400,1.121833,1.121833,NaN,NaN,NaN
3,2025-05-20,1.12358,1.12857,1.12181,1.12844,1.125600,1.125600,NaN,NaN,NaN
4,2025-05-21,1.12821,1.13629,1.12798,1.13310,1.131395,1.131395,NaN,NaN,NaN
5,2025-05-22,1.13250,1.13449,1.12559,1.12786,1.130110,1.130110,NaN,NaN,NaN
6,2025-05-23,1.12788,1.13755,1.12763,1.13640,1.132365,1.132365,NaN,NaN,NaN
7,2025-05-26,1.13626,1.14188,1.13601,1.13868,1.138208,1.138208,NaN,NaN,NaN
8,2025-05-27,1.13828,1.14073,1.13233,1.13270,1.136010,1.136010,NaN,NaN,NaN
9,2025-05-28,1.13254,1.13451,1.12839,1.12894,1.131095,1.131095,NaN,NaN,NaN


In [201]:
def entring_stoploss_profit(row, ohlc_df) :
    if row["sma_1"] > row['sma_14'] and row["sma_11"] > row['sma_144'] :#row["low_x"] <= row["low_20"] :
        return "buy"
    elif row['sma_1'] < row['sma_14'] and row['sma_11'] < row['sma_144'] :#row["high_x"] >= row["high_20"]:
        return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
merged_df1 = pd.merge_asof(ohlc_df, hourly_df, on='time', direction='backward')
merged_df = pd.merge_asof(merged_df1, daily_df, on='time', direction='backward')
merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, ohlc_df = ohlc_df)

# Tag first signal per day in original merged_df
merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
merged_df[['signal', 'time']].dropna()

,signal,time
18760,buy,2025-06-03 00:00:00
21159,buy,2025-06-04 16:00:00
21638,buy,2025-06-05 00:00:00
23077,buy,2025-06-06 00:00:00
24994,buy,2025-06-09 08:00:00
25953,buy,2025-06-10 00:00:00
27392,buy,2025-06-11 00:00:00
28831,buy,2025-06-12 00:00:00
30270,buy,2025-06-13 00:00:00
32187,buy,2025-06-16 08:00:00


In [195]:
merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "low_y": "hourly_low",
    "high_y": "hourly_high",
    "open_y": "hourly_open",
    "close_y": "hourly_close",
    "low": "daily_low",
    "high": "daily_high",
    "open": "daily_open",
    "close": "daily_close"
})


In [196]:
print(merged_df.columns.tolist())
merged_df

['time', 'open', 'high', 'low', 'close', 'hourly_open', 'hourly_high', 'hourly_low', 'hourly_close', 'ohlc_x', 'sma_11', 'sma_144', 'daily_open', 'daily_high', 'daily_low', 'daily_close', 'ohlc_y', 'sma_1', 'sma_14', 'high_20', 'low_20', 'signal', 'is_first_signal']


,time,open,high,low,close,hourly_open,hourly_high,hourly_low,hourly_close,ohlc_x,...,daily_high,daily_low,daily_close,ohlc_y,sma_1,sma_14,high_20,low_20,signal,is_first_signal
0,2025-05-14 23:00:00,1.11674,1.11693,1.11672,1.11687,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
1,2025-05-14 23:01:00,1.11687,1.11687,1.11666,1.11670,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2,2025-05-14 23:02:00,1.11670,1.11680,1.11670,1.11676,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,2025-05-14 23:03:00,1.11676,1.11681,1.11671,1.11673,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
4,2025-05-14 23:04:00,1.11673,1.11674,1.11666,1.11667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53281,2025-07-04 23:52:00,1.17762,1.17766,1.17718,1.17721,1.17743,1.17836,1.17697,1.17736,1.17753,...,1.17876,1.17502,1.17736,1.17669,1.17669,1.164494,1.164075,1.155776,None,NaN
53282,2025-07-04 23:53:00,1.17721,1.17722,1.17719,1.17722,1.17743,1.17836,1.17697,1.17736,1.17753,...,1.17876,1.17502,1.17736,1.17669,1.17669,1.164494,1.164075,1.155776,None,NaN
53283,2025-07-04 23:54:00,1.17722,1.17722,1.17697,1.17713,1.17743,1.17836,1.17697,1.17736,1.17753,...,1.17876,1.17502,1.17736,1.17669,1.17669,1.164494,1.164075,1.155776,None,NaN
53284,2025-07-04 23:55:00,1.17713,1.17716,1.17711,1.17716,1.17743,1.17836,1.17697,1.17736,1.17753,...,1.17876,1.17502,1.17736,1.17669,1.17669,1.164494,1.164075,1.155776,None,NaN


In [197]:
# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        stop_loss_pips = (data['high'] - data["low_20"]) # example SL
        volume = 100000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        stop_loss_pips = (data['high_20'] - data["low_20"]) # example SL
        volume = 100000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data["sma_11"] <= data["sma_144"]:#(data["high"] >= data["high_20"] 
                                             #or data['low_20'] - ((data['high_20'] - data["low_20"]) /2)) :
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and data["sma_11"] >= data["sma_144"]:#(data["low"] <= data["low_20"] 
                                             #or data['high_20'] + ((data['high_20'] - data["low_20"]) /2)) :
            orders.close_trade(trade)

In [198]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(merged_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,buy,100000,2025-06-03 00:00:00,1.1439,2025-06-03 16:00:00,1.1382,0,0,{},-570.0,-7.0,-577.0,-577.0,9423.0
1,closed,EURUSD,buy,100000,2025-06-04 16:00:00,1.14095,2025-06-06 12:00:00,1.14147,0,0,{},52.0,-7.0,45.0,-532.0,9468.0
2,closed,EURUSD,buy,100000,2025-06-09 08:00:00,1.14185,2025-06-09 12:00:00,1.14284,0,0,{},99.0,-7.0,92.0,-440.0,9560.0
3,closed,EURUSD,buy,100000,2025-06-10 00:00:00,1.1417,2025-06-10 04:00:00,1.14243,0,0,{},73.0,-7.0,66.0,-374.0,9626.0
4,closed,EURUSD,buy,100000,2025-06-11 00:00:00,1.14227,2025-06-13 12:00:00,1.15288,0,0,{},1061.0,-7.0,1054.0,680.0,10680.0
5,closed,EURUSD,buy,100000,2025-06-16 08:00:00,1.1536,2025-06-17 16:00:00,1.15503,0,0,{},143.0,-7.0,136.0,816.0,10816.0
6,closed,EURUSD,buy,100000,2025-06-20 00:00:00,1.14918,2025-06-23 00:00:00,1.14562,0,0,{},-356.0,-7.0,-363.0,453.0,10453.0
7,closed,EURUSD,buy,100000,2025-06-23 16:00:00,1.14676,2025-07-02 12:00:00,1.17757,0,0,{},3081.0,-7.0,3074.0,3527.0,13527.0
8,closed,EURUSD,buy,100000,2025-07-03 00:00:00,1.17983,2025-07-03 12:00:00,1.17979,0,0,{},-4.0,-7.0,-11.0,3516.0,13516.0


In [199]:
mm = bt.plot_balance()
display(mm)

In [200]:
evaluate_backtest(bt.trades)

biggest_profit: 3081.0
daily_drawdown: -570.0
max_drawown: -363.0
avg_win: 751.5
avg_loss: -310.0
count_profit_trades: 6
count_loss_trades: 3
Win Rate1: 66.67%
rrr: 2.42


,order_type,profit
0,buy,3579.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,commission,profit_net,profit_cumulative,balance,current_max,drawdown,dayofweek,hourofday,month,year
0,closed,EURUSD,buy,100000,2025-06-03 00:00:00,1.1439,2025-06-03 16:00:00,1.1382,0,0,...,-7.0,-577.0,-577.0,9423.0,-577.0,0.0,1,0,2025-06,2025
1,closed,EURUSD,buy,100000,2025-06-04 16:00:00,1.14095,2025-06-06 12:00:00,1.14147,0,0,...,-7.0,45.0,-532.0,9468.0,-532.0,0.0,2,16,2025-06,2025
2,closed,EURUSD,buy,100000,2025-06-09 08:00:00,1.14185,2025-06-09 12:00:00,1.14284,0,0,...,-7.0,92.0,-440.0,9560.0,-440.0,0.0,0,8,2025-06,2025
3,closed,EURUSD,buy,100000,2025-06-10 00:00:00,1.1417,2025-06-10 04:00:00,1.14243,0,0,...,-7.0,66.0,-374.0,9626.0,-374.0,0.0,1,0,2025-06,2025
4,closed,EURUSD,buy,100000,2025-06-11 00:00:00,1.14227,2025-06-13 12:00:00,1.15288,0,0,...,-7.0,1054.0,680.0,10680.0,680.0,0.0,2,0,2025-06,2025
5,closed,EURUSD,buy,100000,2025-06-16 08:00:00,1.1536,2025-06-17 16:00:00,1.15503,0,0,...,-7.0,136.0,816.0,10816.0,816.0,0.0,0,8,2025-06,2025
6,closed,EURUSD,buy,100000,2025-06-20 00:00:00,1.14918,2025-06-23 00:00:00,1.14562,0,0,...,-7.0,-363.0,453.0,10453.0,816.0,-363.0,4,0,2025-06,2025
7,closed,EURUSD,buy,100000,2025-06-23 16:00:00,1.14676,2025-07-02 12:00:00,1.17757,0,0,...,-7.0,3074.0,3527.0,13527.0,3527.0,0.0,0,16,2025-06,2025
8,closed,EURUSD,buy,100000,2025-07-03 00:00:00,1.17983,2025-07-03 12:00:00,1.17979,0,0,...,-7.0,-11.0,3516.0,13516.0,3527.0,-11.0,3,0,2025-07,2025


ValueError: Plotly Express cannot process wide-form data with columns of different type.